In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
benefitscostsharing = pd.read_csv('../input/health-insurance-marketplace/BenefitsCostSharing.csv')
benefitscostsharing.head()

In [ ]:
plan = benefitscostsharing[['PlanId','IssuerId','BusinessYear','StateCode']]
plan.head()

In [ ]:
plan.drop_duplicates(inplace=True)
len(plan)

In [ ]:
plan.to_csv('Plan.csv',index=False)

In [ ]:
benefits = benefitscostsharing[['PlanId', 'BenefitName', 'BusinessYear', 'CopayInnTier1', 'CopayOutofNet', 'CoinsInnTier1', 'CoinsOutofNet','IsEHB','EHBVarReason','Explanation','IsSubjToDedTier1','IsExclFromInnMOOP','IsExclFromOonMOOP']]
benefits.head()

In [ ]:
benefits['IsEHB'] = benefits['IsEHB'].apply(lambda x: True if isinstance(x, str) else '\\N')

In [ ]:
def process_yn(x):
    if isinstance(x, str):
        if x.lower() == 'no':
            return False
        elif x.lower() == 'yes':
            return True
    else:
        return '\\N'
cols = ['IsSubjToDedTier1','IsExclFromInnMOOP','IsExclFromOonMOOP']

for col in cols:
    benefits[col] = benefits[col].apply(process_yn)

In [ ]:
import re
def process_dollars(x):
    if isinstance(x, str):
        if 'No Charge' in x:
            return 0
        else:
            temp = re.findall(r'\d+', x)
            if len(temp) > 0:
                return int(temp[0])
            else:
                return '\\N'
    else:
        return '\\N'
    
def process_condition(x):
    if isinstance(x, str):
        if 'No Charge' in x:
            temp = x.split("No Charge",1)[1].strip()
            if temp == '':
                return '\\N'
            else:
                return temp
        else:
            temp = ''.join(c for c in x if c.isalpha() or c == ' ').strip()
            if temp == '':
                return '\\N'
            else:
                return temp
    else:
        return '\\N'
    
benefits['CopayInnTier1Amount'] = benefits['CopayInnTier1'].apply(process_dollars)
benefits['CopayOutofNetAmount'] = benefits['CopayOutofNet'].apply(process_dollars)
benefits['CopayInnTier1Condition'] = benefits['CopayInnTier1'].apply(process_condition)
benefits['CopayOutofNetCondition'] = benefits['CopayOutofNet'].apply(process_condition)

In [ ]:
benefits.drop_duplicates(inplace=True)

In [ ]:
cleaned_benefits = benefits[['PlanId','BenefitName','BusinessYear','CopayInnTier1Amount','CopayOutofNetAmount','CopayInnTier1Condition','CopayOutofNetCondition','CoinsInnTier1','CoinsOutofNet','Explanation','IsSubjToDedTier1','IsExclFromInnMOOP','IsExclFromOonMOOP']]
filtered = ((cleaned_benefits['CopayInnTier1Amount'] != '\\N') & (cleaned_benefits['CopayOutofNetAmount'] != '\\N'))
cleaned_benefits = cleaned_benefits[filtered]

In [ ]:
cleaned_benefits['Explanation'] = cleaned_benefits['Explanation'].apply(lambda x: x if isinstance(x, str) else '\\N')

In [ ]:
cleaned_benefits['CoinsInnTier1'] = cleaned_benefits['CoinsInnTier1'].apply(lambda x: x.replace('Coinsurance',''))
cleaned_benefits['CoinsInnTier1'] = cleaned_benefits['CoinsInnTier1'].apply(lambda x: x.replace('No Charge','0%'))

In [ ]:
cleaned_benefits['CoinsOutofNet'] = cleaned_benefits['CoinsOutofNet'].apply(lambda x: x.replace('Coinsurance',''))
cleaned_benefits['CoinsOutofNet'] = cleaned_benefits['CoinsOutofNet'].apply(lambda x: x.replace('No Charge','0%'))

In [ ]:
def process_yn_new(x):
    if x == True:
        return 1
    elif x == False:
        return 0
    else:
        return '\\N'
    
cols = ['IsSubjToDedTier1','IsExclFromInnMOOP','IsExclFromOonMOOP']

for col in cols:
    cleaned_benefits[col] = cleaned_benefits[col].apply(process_yn_new)

In [ ]:
cleaned_benefits['CopayInnTier1Amount'].unique()

In [ ]:
cleaned_benefits.to_csv('Benefits.csv',index=False)

In [ ]:
cleaned_benefits.head()